To start this Jupyter Dash app, please run all the cells below. Then, click on the **temporary** URL at the end of the last cell to open the app.

In [1]:
!pip install -q jupyter-dash==0.3.0rc1 dash-bootstrap-components transformers

In [3]:
!pip install torch

In [5]:
import time

import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import torch
from transformers import MarianMTModel, MarianTokenizer
import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Choose device and load model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
model.eval();

Device: cuda


Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ROMANCE and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Create app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container(
    fluid=True,
    children=[
        html.H1("Dash Neural Machine Translation"),
        html.Hr(),
        dbc.Spinner(
            dbc.Row(
                [
                    dbc.Col(dbc.Button("Translate", id="button-translate"), width=2),
                    dbc.Col(
                        html.Div(id="time-output", style={"margin-top": "8px"}),
                        width=10,
                    ),
                ],
                style={"margin-bottom": "15px"},
            )
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dbc.InputGroup(
                            [
                                dbc.InputGroupAddon(
                                    "Source Language", addon_type="prepend"
                                ),
                                dbc.Select(
                                    id="source-language",
                                    options=[{"label": "English", "value": "en"}],
                                    value="en",
                                ),
                            ]
                        ),
                        dbc.Textarea(
                            id="source-text",
                            style={"margin-top": "15px", "height": "65vh"},
                        ),
                    ]
                ),
                dbc.Col(
                    [
                        dbc.InputGroup(
                            [
                                dbc.InputGroupAddon(
                                    "Target Language", addon_type="prepend"
                                ),
                                dbc.Select(
                                    id="target-language",
                                    options=[
                                        {
                                            "label": v.replace(">>", "").replace(
                                                "<<", ""
                                            ),
                                            "value": v,
                                        }
                                        for v in tokenizer.supported_language_codes
                                    ],
                                    value=">>fr<<",
                                ),
                            ]
                        ),
                        dbc.Textarea(
                            id="target-text",
                            style={"margin-top": "15px", "height": "65vh"},
                        ),
                    ]
                ),
            ]
        ),
    ],
)

In [5]:
@app.callback(
    [Output("target-text", "value"), Output("time-output", "children")],
    [
        Input("button-translate", "n_clicks"),
        Input("source-language", "value"),
        Input("target-language", "value"),
    ],
    [State("source-text", "value")],
)
def translate(n_clicks, src_lang, tgt_lang, src_text):
    if src_text is None or src_text == "":
        return "", "Did not run."

    t0 = time.time()

    # Create the batched input
    text_in = [
        tgt_lang + " " + sent.replace("\n", " ") for sent in sent_tokenize(src_text)
    ]
    batch = tokenizer.prepare_translation_batch(text_in)
    for k in batch:
        batch[k] = batch[k].to(device)

    # Run the model and decode the output
    translated = model.generate(**batch)
    tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    t1 = time.time()
    time_output = f"Translated on {device} in {t1-t0:.2f}s"

    return " ".join(tgt_text), time_output

In [12]:
app.run_server(mode='external', port='7007')

Dash app running on http://127.0.0.1:7007/
